In [1]:
import torch
from transformers import (
    AutoModelForCausalLM,
    LlamaTokenizerFast,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig,
)
from peft import (
    LoraConfig,
    get_peft_model,
)

2024-04-18 10:37:18.706317: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-18 10:37:18.814490: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-18 10:37:19.342040: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2024-04-18 10:37:19.342096: W tensorflow/compiler/xla/stream_exe

In [2]:
# https://huggingface.co/meta-llama/Llama-2-7b-hf
base_model = "meta-llama/Llama-2-7b-hf"
# base_model = "TinyPixel/Llama-2-7B-bf16-sharded"

In [3]:
# # Fine-tuned model
# new_model = "llama-2-7b-hf-fine-tuned-test1"

In [3]:
# QLoRA 모델을 사용하기 위한 설정

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16 
)



In [4]:
# 라마2 모델 불러오기
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
# 토크나이저 로드
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false" # 토크나이저 병렬처리 방지(오류 방지)
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = 'true' # __cell__ 오류 방지

tokenizer = LlamaTokenizerFast.from_pretrained(
    base_model,
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token # 패딩 토큰을 문장의 끝으로 설정 </s>
tokenizer.padding_side = "right" # 패딩을 문장 뒤에 추가

In [8]:
# 학습 양식
import json
from datasets import load_dataset
file_name = 'preprocess/dataset3/new_ratings.json'

In [9]:
import json

# JSON 파일 불러오기
input_file = file_name
with open(input_file, 'r') as f:
    data = json.load(f)

# 불러온 데이터 확인
# print(data)

In [10]:
# print(data[1])

print(data[0]["ratings"]) # 영화 평점들 정보 모음
print(type(data[0]["ratings"])) 

print(data[0]["ratings"][0]) # 평점 1개
print(type(data[0]["ratings"][0])) 
print(data[0]["order"])

[{'imdbId': 'tt114709', 'rating': 4, 'title': 'Toy Story (1995)', 'genres': 'Adventure|Animation|Children|Comedy|Fantasy', 'timestamp': 964982703}, {'imdbId': 'tt113228', 'rating': 4, 'title': 'Grumpier Old Men (1995)', 'genres': 'Comedy|Romance', 'timestamp': 964981247}, {'imdbId': 'tt113277', 'rating': 4, 'title': 'Heat (1995)', 'genres': 'Action|Crime|Thriller', 'timestamp': 964982224}, {'imdbId': 'tt114369', 'rating': 5, 'title': 'Seven (a.k.a. Se7en) (1995)', 'genres': 'Mystery|Thriller', 'timestamp': 964983815}, {'imdbId': 'tt114814', 'rating': 5, 'title': 'Usual Suspects, The (1995)', 'genres': 'Crime|Mystery|Thriller', 'timestamp': 964982931}, {'imdbId': 'tt116367', 'rating': 3, 'title': 'From Dusk Till Dawn (1996)', 'genres': 'Action|Comedy|Horror|Thriller', 'timestamp': 964982400}, {'imdbId': 'tt115734', 'rating': 5, 'title': 'Bottle Rocket (1996)', 'genres': 'Adventure|Comedy|Crime|Romance', 'timestamp': 964980868}, {'imdbId': 'tt112573', 'rating': 4, 'title': 'Braveheart (1

In [11]:
instruction = '''
I'll determine the priority of the movie recommendation list based on the user's movie rating information. I'll return the recommended list of movies sorted according to priority imdb_id, and I'll output the metadata for the corresponding imdb_id. It must return in JSON format.
'''

In [12]:
# 데이터 매핑 함수 정의
def map_data_to_format(example):

    user_ratings_str = json.dumps(example['ratings'], ensure_ascii=False)
    order_list_str = json.dumps(example["order"], ensure_ascii=False)
    
    text = (
        f"###instruction:\n{instruction}\n\n"
        f"user_ratings_info:\n{user_ratings_str}\n\n"
        f"###order_list:\n{order_list_str}\n\n"
    )
    
    # completion은 모델이 생성해야 할 예상 출력을 포함함
    # 여기서는 우선순위에 따라 정렬된 영화 imdb_id의 리스트를 반환합니다.
    completion = f"{{\"order_list\": {order_list_str}}}"
    
    return {'text': text, 'completion': completion}

In [13]:
# 데이터셋 로드
data = load_dataset('json', data_files=file_name, split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [14]:
print(data)

Dataset({
    features: ['ratings', 'userId', 'order'],
    num_rows: 10084
})


In [15]:
# print(data['ratings'])


In [16]:
# print(data['order']) # [['tt109370', 'tt114287', 'tt114369', 'tt114814', 'tt115734', 'tt114709', 'tt113277', 'tt112573', 'tt113228', 'tt116367'], ...]

In [17]:
# print(data['userId']) # [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, ...]

In [18]:
# 데이터 매핑 적용
mapped_data = data.map(map_data_to_format)

Map:   0%|          | 0/10084 [00:00<?, ? examples/s]

In [19]:
print(mapped_data)


Dataset({
    features: ['ratings', 'userId', 'order', 'text', 'completion'],
    num_rows: 10084
})


In [20]:
# 데이터셋 분할
split_data = mapped_data.train_test_split(test_size=0.1)  # 10%를 테스트셋으로 사용

train_set = split_data['train']
eval_set = split_data['test']

train_set = train_set.map(lambda samples: tokenizer(samples["text"], padding=True, truncation=True, return_tensors="pt"), batched=True)
eval_set = eval_set.map(lambda samples: tokenizer(samples["text"], padding=True, truncation=True, return_tensors="pt"), batched=True)

# lora 파라미터 설정
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)


Map:   0%|          | 0/9075 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/1009 [00:00<?, ? examples/s]

In [22]:
print(train_set[3]["text"])
# print(train_set[20]["text"])
print(train_set[120]["text"])

###instruction:

I'll determine the priority of the movie recommendation list based on the user's movie rating information. I'll return the recommended list of movies sorted according to priority imdb_id, and I'll output the metadata for the corresponding imdb_id. It must return in JSON format.


user_ratings_info:
[{"genres": "Action|Adventure|Thriller", "imdbId": "tt381061", "rating": 4, "timestamp": 1521467766, "title": "Casino Royale (2006)"}, {"genres": "Animation|Children|Drama", "imdbId": "tt382932", "rating": 5, "timestamp": 1521398256, "title": "Ratatouille (2007)"}, {"genres": "Action|Crime|Thriller", "imdbId": "tt440963", "rating": 4, "timestamp": 1521467753, "title": "Bourne Ultimatum, The (2007)"}, {"genres": "Action|Adventure|Sci-Fi", "imdbId": "tt371746", "rating": 5, "timestamp": 1513712154, "title": "Iron Man (2008)"}, {"genres": "Adventure|Animation|Children|Romance|Sci-Fi", "imdbId": "tt910970", "rating": 5, "timestamp": 1513712149, "title": "WALL·E (2008)"}, {"genre

In [23]:
print(eval_set[3]["text"])
# print(eval_set[20]["text"])
print(eval_set[120]["text"])

###instruction:

I'll determine the priority of the movie recommendation list based on the user's movie rating information. I'll return the recommended list of movies sorted according to priority imdb_id, and I'll output the metadata for the corresponding imdb_id. It must return in JSON format.


user_ratings_info:
[{"genres": "Drama|Sci-Fi", "imdbId": "tt118884", "rating": 4, "timestamp": 1034135578, "title": "Contact (1997)"}, {"genres": "Action|Adventure|Sci-Fi|Thriller", "imdbId": "tt120177", "rating": 2, "timestamp": 1034138273, "title": "Spawn (1997)"}, {"genres": "Drama|Mystery|Romance|Thriller", "imdbId": "tt118883", "rating": 5, "timestamp": 1034138875, "title": "Conspiracy Theory (1997)"}, {"genres": "Crime|Film-Noir|Mystery|Thriller", "imdbId": "tt119488", "rating": 4, "timestamp": 1034136028, "title": "L.A. Confidential (1997)"}, {"genres": "Adventure|Drama|War", "imdbId": "tt120102", "rating": 4, "timestamp": 1034138678, "title": "Seven Years in Tibet (1997)"}, {"genres": 

In [24]:
print(train_set[3]["completion"])
# print(train_set[20]["completion"])
print(train_set[120]["completion"])

{"order_list": ["tt382932", "tt371746", "tt1049413", "tt910970", "tt1132620", "tt1136608", "tt381061", "tt440963", "tt1010048", "tt361748"]}
{"order_list": ["tt325805", "tt335266", "tt257076", "tt320691", "tt266489", "tt327137", "tt322330", "tt329101", "tt313911", "tt263757"]}


In [25]:
model = get_peft_model(model, peft_params)

# prameter
epochs = 3 # 10
batch_size = 2
lr = 2e-4

training_params = TrainingArguments(
    output_dir="models",
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=16,
    optim="adamw_torch",
    save_strategy="epoch",
    evaluation_strategy="steps",
    logging_strategy="steps",
    eval_steps=20,
    logging_steps=20,
    learning_rate=lr,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
    dataloader_num_workers=1,
)


In [22]:
# ! wandb login --relogin


In [26]:
import transformers
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

trainer = Trainer(
    model=model,
    args=training_params,
    train_dataset=train_set,
    eval_dataset=eval_set,
    tokenizer=tokenizer,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

trainer.train()

/home/gpuadmin/anaconda3/envs/llm_test/lib/python3.8/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


  0%|          | 0/849 [00:00<?, ?it/s]

{'loss': 0.8849, 'grad_norm': 0.07385071367025375, 'learning_rate': 0.00015384615384615385, 'epoch': 0.07}


  0%|          | 0/127 [00:00<?, ?it/s]

{'eval_loss': 0.7833625674247742, 'eval_runtime': 266.6675, 'eval_samples_per_second': 3.784, 'eval_steps_per_second': 0.476, 'epoch': 0.07}
{'loss': 0.6164, 'grad_norm': 0.0474424809217453, 'learning_rate': 0.00019985723465913057, 'epoch': 0.14}


...

  0%|          | 0/127 [00:00<?, ?it/s]

{'eval_loss': 0.5095124840736389, 'eval_runtime': 257.6794, 'eval_samples_per_second': 3.916, 'eval_steps_per_second': 0.493, 'epoch': 2.89}
{'loss': 0.5102, 'grad_norm': 0.24122254550457, 'learning_rate': 5.900820203481594e-08, 'epoch': 2.96}


  0%|          | 0/127 [00:00<?, ?it/s]

{'eval_loss': 0.5094553232192993, 'eval_runtime': 265.8515, 'eval_samples_per_second': 3.795, 'eval_steps_per_second': 0.478, 'epoch': 2.96}
{'train_runtime': 33845.5485, 'train_samples_per_second': 0.804, 'train_steps_per_second': 0.025, 'train_loss': 0.5370609004871022, 'epoch': 2.99}


TrainOutput(global_step=849, training_loss=0.5370609004871022, metrics={'train_runtime': 33845.5485, 'train_samples_per_second': 0.804, 'train_steps_per_second': 0.025, 'train_loss': 0.5370609004871022, 'epoch': 2.99})

In [27]:
from tensorboard import notebook
log_dir = "./models" 
notebook.start("--logdir {} --port 4000".format(log_dir))

In [29]:
instruction = "I'll determine the priority of the movie recommendation list based on the user's movie rating information. I'll return the recommended list of movies sorted according to priority imdb_id, and I'll output the metadata for the corresponding imdb_id. It must return in JSON format."

user_rating_str = [{"genres": "Drama|Sci-Fi", "imdbId": "tt118884", "rating": 4, "timestamp": 1034135578, "title": "Contact (1997)"}, {"genres": "Action|Adventure|Sci-Fi|Thriller", "imdbId": "tt120177", "rating": 2, "timestamp": 1034138273, "title": "Spawn (1997)"}, {"genres": "Drama|Mystery|Romance|Thriller", "imdbId": "tt118883", "rating": 5, "timestamp": 1034138875, "title": "Conspiracy Theory (1997)"}, {"genres": "Crime|Film-Noir|Mystery|Thriller", "imdbId": "tt119488", "rating": 4, "timestamp": 1034136028, "title": "L.A. Confidential (1997)"}, {"genres": "Adventure|Drama|War", "imdbId": "tt120102", "rating": 4, "timestamp": 1034138678, "title": "Seven Years in Tibet (1997)"}, {"genres": "Drama|Mystery|Thriller", "imdbId": "tt119174", "rating": 2, "timestamp": 1034139049, "title": "Game, The (1997)"}, {"genres": "Crime|Drama|Mystery", "imdbId": "tt120399", "rating": 1, "timestamp": 1034139135, "title": "U Turn (1997)"}, {"genres": "Comedy|Drama|Romance", "imdbId": "tt118842", "rating": 2, "timestamp": 1034136119, "title": "Chasing Amy (1997)"}, {"genres": "Horror|Mystery|Thriller", "imdbId": "tt119345", "rating": 3, "timestamp": 1034139198, "title": "I Know What You Did Last Summer (1997)"}, {"genres": "Drama|Mystery|Thriller", "imdbId": "tt118971", "rating": 5, "timestamp": 1034139094, "title": "The Devil's Advocate (1997)"}]

order_list_str=["tt118971", "tt118883", "tt120102", "tt119488", "tt118884", "tt119345", "tt119174", "tt120177", "tt118842", "tt120399"]

In [30]:
# ###instruction:
# {instruction}

text = f'''
###instruction:
{instruction}


user_ratings_info:
{user_rating_str}


'''
inputs = tokenizer(text, return_tensors="pt").to("cuda")
outputs = model.generate(
    input_ids=inputs["input_ids"].to("cuda"), 
    attention_mask=inputs["attention_mask"], 
    max_new_tokens=256,
    early_stopping=True,
    pad_token_id=tokenizer.eos_token_id
)
output = tokenizer.decode(outputs[0])
print(output)


<s> 
###instruction:
I'll determine the priority of the movie recommendation list based on the user's movie rating information. I'll return the recommended list of movies sorted according to priority imdb_id, and I'll output the metadata for the corresponding imdb_id. It must return in JSON format.


user_ratings_info:
[{'genres': 'Drama|Sci-Fi', 'imdbId': 'tt118884', 'rating': 4, 'timestamp': 1034135578, 'title': 'Contact (1997)'}, {'genres': 'Action|Adventure|Sci-Fi|Thriller', 'imdbId': 'tt120177', 'rating': 2, 'timestamp': 1034138273, 'title': 'Spawn (1997)'}, {'genres': 'Drama|Mystery|Romance|Thriller', 'imdbId': 'tt118883', 'rating': 5, 'timestamp': 1034138875, 'title': 'Conspiracy Theory (1997)'}, {'genres': 'Crime|Film-Noir|Mystery|Thriller', 'imdbId': 'tt119488', 'rating': 4, 'timestamp': 1034136028, 'title': 'L.A. Confidential (1997)'}, {'genres': 'Adventure|Drama|War', 'imdbId': 'tt120102', 'rating': 4, 'timestamp': 1034138678, 'title': 'Seven Years in Tibet (1997)'}, {'genre